<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #03



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#ramdom)


<br>
<br>


<a id='prob'></a>
## Problem #03

<br>
Facilities <br>
$|I| = 25$ 
<br>
<br>
Clients <br>
$|J| = 50$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [1]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/ORcap104.txt'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))



# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

Using license file /Users/danielcinalli/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-27
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1300 rows, 1275 columns and 3750 nonzeros
Model fingerprint: 0x379392c7
Variable types: 1250 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3132764.7125
Presolve removed 1116 rows and 1107 columns
Presolve time: 0.02s
Presolved: 184 rows, 168 columns, 454 nonzeros
Found heuristic solution: objective 1090882.1500
Variable types: 152 continuous, 16 integer (16 binary)

Root relaxation: objective 9.289418e+05, 69 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

In [3]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('UFLP_03_Simplex.lp')


Build a warehouse at location 11.

Build a warehouse at location 13.

Build a warehouse at location 18.

Build a warehouse at location 24.

Client 1 receives 100.0 % of its demand  from Warehouse 11 .

Client 2 receives 100.0 % of its demand  from Warehouse 11 .

Client 3 receives 100.0 % of its demand  from Warehouse 11 .

Client 4 receives 100.0 % of its demand  from Warehouse 18 .

Client 5 receives 100.0 % of its demand  from Warehouse 11 .

Client 6 receives 100.0 % of its demand  from Warehouse 11 .

Client 7 receives 100.0 % of its demand  from Warehouse 18 .

Client 8 receives 100.0 % of its demand  from Warehouse 11 .

Client 9 receives 100.0 % of its demand  from Warehouse 11 .

Client 10 receives 100.0 % of its demand  from Warehouse 11 .

Client 11 receives 100.0 % of its demand  from Warehouse 11 .

Client 12 receives 100.0 % of its demand  from Warehouse 11 .

Client 13 receives 100.0 % of its demand  from Warehouse 11 .

Client 14 receives 100.0 % of its demand  from Wa

<br>
<br>

<a id='xxx'></a>
### Heuristica



In [4]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)



#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23, 24]
2351240.5374999987


In [5]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 1.

Build a warehouse at location 2.

Build a warehouse at location 3.

Build a warehouse at location 5.

Build a warehouse at location 6.

Build a warehouse at location 7.

Build a warehouse at location 8.

Build a warehouse at location 9.

Build a warehouse at location 10.

Build a warehouse at location 11.

Build a warehouse at location 12.

Build a warehouse at location 13.

Build a warehouse at location 14.

Build a warehouse at location 15.

Build a warehouse at location 16.

Build a warehouse at location 17.

Build a warehouse at location 18.

Build a warehouse at location 20.

Build a warehouse at location 21.

Build a warehouse at location 22.

Build a warehouse at location 24.

Build a warehouse at location 25.

Client 1 receives 100% of its demand  from Warehouse 11 .

Client 2 receives 100% of its demand  from Warehouse 14 .

Client 3 receives 100% of its demand  from Warehouse 2 .

Client 4 receives 100% of its demand  from Warehouse 9 .

Cli

In [6]:
len(facs)

22